# COLLECT FUNDAMENTAL DATA FROM ALPHA VANTAGE

In [1]:
import pandas as pd
import requests

In [2]:
api_key = 'EEAI5CE759SUIP6I'
features_cols = ['fiscalDateEnding','totalRevenue','costOfRevenue','sellingGeneralAndAdministrative','researchAndDevelopment','depreciation','interestExpense','totalCurrentLiabilities','incomeTaxExpense','netIncome','commonStockSharesOutstanding','cashAndCashEquivalentsAtCarryingValue','cashAndShortTermInvestments','operatingCashflow','totalLiabilities','inventory','currentNetReceivables','propertyPlantEquipment','capitalExpenditures','longTermInvestments','totalShareholderEquity','longTermDebt','retainedEarnings','dividendPayoutCommonStock','paymentsForRepurchaseOfCommonStock','treasuryStock','currentLongTermDebt']
price_cols = ['open','high','low','close','volume']
DOW_30_TICKER = ['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']

In [3]:
def collect_fundamental(ticket):
    # Download income statement
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={ticket}&apikey={api_key}'
    r = requests.get(url)
    data_is = r.json()

    # Download balance sheet
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={ticket}&apikey={api_key}'
    r = requests.get(url)
    data_bs = r.json()

    # Download cash flow
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={ticket}&apikey={api_key}'
    r = requests.get(url)
    data_cf = r.json()

    df_is = pd.json_normalize(data_is['annualReports'])
    df_bs = pd.json_normalize(data_bs['annualReports'])
    df_cf = pd.json_normalize(data_cf['annualReports'])

    merged_df = df_is.merge(df_bs).merge(df_cf)
    merged_df['tic'] = ticket
    merged_df = merged_df[['tic']+features_cols]
    merged_df['fiscalDateEnding'] = pd.to_datetime(merged_df.fiscalDateEnding,format='mixed')

    return merged_df

Check reach download limitation

In [4]:
url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol=GS&apikey=GUKB8WJD0YFELJD3'
r = requests.get(url)
data_is = r.json()
data_is

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}

In [ ]:
fund_data = pd.DataFrame()
tickets = DOW_30_TICKER[7:15]
for ticket in tickets:
    df_fund = collect_fundamental(ticket)
    fund_data = pd.concat([fund_data, df_fund], ignore_index=True)

In [ ]:
fund_data.to_csv('dow30.csv')